In [1]:
from collections import Counter

# Card ranks and suits
ranks = '23456789TJQKA'
suits = 'shdc'

def get_rank(card):
    return card[0]

def get_suit(card):
    return card[1]

def get_hand_rank(hand):
    rank_values = {rank.lower(): i for i, rank in enumerate(ranks)}
    ranks_count = [get_rank(card).lower() for card in hand]
    suits_count = [get_suit(card) for card in hand]
    rank_counts = Counter(ranks_count)
    suit_counts = Counter(suits_count)

    straight = False
    flush = False

    # Check for straight
    sorted_ranks = sorted(ranks_count, key=lambda x: rank_values[x], reverse=True)
    highest_rank = rank_values[sorted_ranks[0]]
    lowest_rank = rank_values[sorted_ranks[-1]]
    if len(rank_counts) == 5 and (highest_rank - lowest_rank == 4):
        straight = True

    # Check for flush
    if len(suit_counts) == 1:
        flush = True

    # Check for straight flush
    if straight and flush:
        return 9, highest_rank

    # Check for four of a kind
    if len(rank_counts) == 2 and 4 in rank_counts.values():
        for rank, count in rank_counts.items():
            if count == 4:
                return 8, rank_values[rank]

    # Check for full house
    if len(rank_counts) == 2 and 3 in rank_counts.values():
        for rank, count in rank_counts.items():
            if count == 3:
                return 7, rank_values[rank]

    # Check for flush
    if flush:
        return 6, highest_rank

    # Check for straight
    if straight:
        return 5, highest_rank

    # Check for three of a kind
    if 3 in rank_counts.values():
        for rank, count in rank_counts.items():
            if count == 3:
                return 4, rank_values[rank]

    # Check for two pair
    if len(rank_counts) == 3 and 2 in rank_counts.values():
        pairs = []
        for rank, count in rank_counts.items():
            if count == 2:
                pairs.append(rank_values[rank])
        return 3, max(pairs)

    # Check for one pair
    if len(rank_counts) == 4 and 2 in rank_counts.values():
        for rank, count in rank_counts.items():
            if count == 2:
                return 2, rank_values[rank]

    # High card
    return 1, highest_rank

def compare_hands(player_hand, other_hands, flopped_cards):
    best_rank = get_hand_rank(player_hand + flopped_cards)
    winners = [player_hand]

    for hand in other_hands:
        rank = get_hand_rank(hand+flopped_cards)
        if rank > best_rank:
            best_rank = rank
            winners = [hand]
        elif rank == best_rank:
            winners.append(hand)

    return winners, best_rank


def eval_hands_QUICK(player_hand, other_hands, flopped_cards):
    best_rank = get_hand_rank(player_hand + flopped_cards)
    winners = [player_hand]
    draw_possible = False
    for hand in other_hands:
        rank = get_hand_rank(hand+flopped_cards)
        if rank > best_rank:
            return -1
            best_rank = rank
            winners = [hand]
        elif rank == best_rank:
            draw_possible = True
    
    if draw_possible == True:
        return 0
    return 1

def pick_best_hand(winners):
    best_hand = winners[0]
    best_rank = get_hand_rank(best_hand)

    for hand in winners[1:]:
        rank = get_hand_rank(hand)
        if rank > best_rank:
            best_hand = hand
            best_rank = rank

    return best_hand



# Example usage:
player_hand = ['Kc', 'Jc']
other_hands = [['Ah', 'Qh'],['As', 'Kd'] , ['8s', '8h']]



player_hand = ['kc', 'Jc']
other_hands = [['Ah', 'Qh'],['Kc', 'Js'] , ['8s', '8h']]


flopped_cards = ['Ts', 'Js', 'Qs', 'Ks', '9s']

# winners, best_rank = compare_hands(player_hand, other_hands, flopped_cards)
# print("Winning hand(s):")
# for hand in winners:
#     print(hand)

# best_hand = pick_best_hand(winners)
# print("Best hand:", best_hand)



eval_hands_QUICK(player_hand, other_hands, flopped_cards)

0

In [14]:
import random

# USER INPUTS
my_hand = ["4s", "2h"]

def sample_flop_and_player_hands(all_cards, n_players ):
    hands = []
    for _ in range(n_players):
        hand = [all_cards.pop(random.randint(0, len(all_cards)-1)) for _ in range(2)]
        hands.append(hand)
    flop = [all_cards.pop(random.randint(0, len(all_cards)-1)) for _ in range(5)]
    return hands, flop


def subtract(all_cards,known_flops):
    filtered = []
    for card in enumerate(all_cards):
        isFlop = False
        for x in known_flops:
            if card == x:
                isFlop = True
                break
        if not isFlop:
            filtered.append(card)

    return filtered


def get_deck(my_hand):
    full_deck = []
    for suit in ['s','c','h','d']:
        for num in ["2","3","4","5","6","7","8","9","t","j","q","k","a"]:
            if num+suit not in my_hand:
                full_deck.append(num+suit)
    return full_deck


from tqdm import tqdm
#pick cards for flopp and other players
my_hand = ["ks", "kc"]


#calculate values for my hand:
all_cards = get_deck(my_hand)
n_other_players = 5

total_games = 0
num_wins = 0
num_draws = 0
num_losses = 0

num_iterations = 10000000
known_flops = []


all_cards = subtract(all_cards, known_flops)
for i, x in enumerate(range(num_iterations)):
    all_cards_copy = all_cards.copy()
    hands, flop = sample_flop_and_player_hands(all_cards_copy,n_other_players )
    res = eval_hands_QUICK(my_hand, hands, flop)
    if res ==1:
        num_wins +=1
    elif res == 0:
        num_draws +=1 
    elif res == -1:
        num_losses +=1
    if i %1000 == 0:
        r_avg = num_wins/(num_wins+num_losses)
        print(f"Running Average: {r_avg}", end="\r")



TypeError: 'int' object is not subscriptable